In [3]:
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd

!pip install folium
import folium
import  json  # library to handle JSON files# librar 

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

distributed 1.21.8 requires msgpack, which is not installed.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2018.8.24          |        py36_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         220 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.17.0-py_0      conda-forge

The following packages will be UPDATED:

    certifi:       2018.8.24-py36_1 conda-forge --> 2018.8.24-py36_1001 conda-forge


geopy-1.17.0         | 4

In [9]:
# Bucharest
wiki = "https://en.wikipedia.org/wiki/List_of_Bucharest_metro_stations"

website_url = requests.get(wiki).text
soup = BeautifulSoup(website_url,'lxml')

my_table = soup.find('table',{'class':'wikitable sortable'})
#print(my_table)

station = []
stationro = []

for row in my_table.find_all('tr')[1:]:
    station_cell = row.find_all('td')[0] 
    station.append(station_cell.text)

for row in my_table.find_all('tr')[1:]:
    stationro_cell = row.find_all('td')[1]
    stationro.append(stationro_cell.text)


dfm = pd.DataFrame(
{'Station': station,
'Stationro': stationro
})

dfm['Station']=dfm['Station'].str.replace('\n','')
dfm['Stationro']=dfm['Stationro'].str.replace('\n','')

# Clean data 
dfm = dfm.drop_duplicates(subset=['Station'], keep=False)
dfm['Address'] =['Меtrou, '] + dfm['Stationro']
dfm.set_index('Address', inplace=True)
dfm.head(5)

,Station,Stationro
Address,,
"Меtrou, Piaţa Victoriei 1 / Piaţa Victoriei 2 (Victory Square)","Line 1, Line 2",Piaţa Victoriei 1 / Piaţa Victoriei 2 (Victory...
"Меtrou, Piaţa Unirii 1 / Piaţa Unirii 2 (Union Square)","Line 1, Line 2, Line 3",Piaţa Unirii 1 / Piaţa Unirii 2 (Union Square)
"Меtrou, Pantelimon",Line 1*,Pantelimon


In [10]:
#my Google API key
API_key = 'My API_key'

In [22]:
#Find longitute, latitude and administrativ regions of Bucharest metro stations
for pcode in dfm.index:
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, pcode)

    response = requests.get(url).json() # get response
    try:
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        address_data = response['results'][0]['plus_code'] #get administrativ district
        latitude = geographical_data['lat']
        longitude = geographical_data['lng']
        address = address_data['compound_code']
       
        dfm.loc[dfm.index == pcode, 'Latitute']= latitude
        dfm.loc[dfm.index == pcode, 'Longitude']= longitude
        dfm.loc[dfm.index == pcode, 'adresa']= address
        
        
    except:
        pass

dfm.head(1)

,Station,Stationro
Address,,
"Меtrou, Piaţa Victoriei 1 / Piaţa Victoriei 2 (Victory Square)","Line 1, Line 2",Piaţa Victoriei 1 / Piaţa Victoriei 2 (Victory...


In [23]:
dfm

,Station,Stationro
Address,,
"Меtrou, Piaţa Victoriei 1 / Piaţa Victoriei 2 (Victory Square)","Line 1, Line 2",Piaţa Victoriei 1 / Piaţa Victoriei 2 (Victory...
"Меtrou, Piaţa Unirii 1 / Piaţa Unirii 2 (Union Square)","Line 1, Line 2, Line 3",Piaţa Unirii 1 / Piaţa Unirii 2 (Union Square)
"Меtrou, Pantelimon",Line 1*,Pantelimon


In [35]:
CLIENT_ID = 'ABLICKN55QXJ4YZHEX2CK5NIA0VPYBF2DQG2FAQBGOMCGPWZ'
CLIENT_SECRET = '5U33ROG3PVFVKMVWTO3F4Y01NUMGK14HZVSVIX03WXJPZZPA'
VERSION = '21080210'

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [1]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# create map Cluj
map_clusters = folium.Map(location=[40.177200, 44.503490], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(# create map Cluj
map_clusters = folium.Map(location=[40.177200, 44.503490], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bucharest_merged['Latitute'], bucharest_merged['Longitude'], bucharest_merged['Address'], bucharest_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters_merged['Latitute'], bucharest_merged['Longitude'], bucharest_merged['Address'], bucharest_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters